In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [2]:
import numpy as np
import geopandas as gpd
import rasterio
import xarray as xa
import rioxarray as rio
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from pathlib import Path
import pathlib
from tqdm import tqdm
from scipy.ndimage import binary_dilation

import netCDF4

from coralshift.utils import file_ops, directories
from coralshift.dataloading import bathymetry
from coralshift.processing import data

# Climate data imports

In [19]:
def read_nc_path(nc_file_path: Path | str) -> xa.DataArray:
    """TODO: docstring"""
    # return xa.load_dataset(nc_file_path)
    file_read = xa.open_dataset(nc_file_path, engine="netcdf4")
    # file_read = netCDF4.Dataset(nc_file_path, "r")
    # file_read.close()
    return file_read


def read_ncs_from_dir(dir_path: Path | str) -> dict:
    """TODO: docstring
    Could also make into a more generic function which reads in different file types in correct way"""
    nc_files_list = file_ops.return_list_filepaths(dir_path, ".nc")

    nc_arrays_dict = {}
    for nc_path in tqdm(nc_files_list):
        path_end = file_ops.get_n_last_subparts_path(nc_path, 1)
        nc_filename = file_ops.remove_suffix(str(path_end))
        array = read_nc_path(nc_path).rio.write_crs("epsg:4326")

        nc_arrays_dict[nc_filename] = array

    return nc_arrays_dict



In [20]:
out = read_ncs_from_dir("/Volumes/MRes Drive/global_ocean_reanalysis/climatology_monthly_means")

100%|██████████| 12/12 [00:01<00:00,  8.91it/s]


In [23]:
out["mercatorglorys12v1_gl12_mean_1993_2016_04"]

<xarray.Dataset>
Dimensions:      (time: 1, longitude: 4320, latitude: 2041, depth: 50)
Coordinates:
  * time         (time) datetime64[ns] 1993-04-16
  * longitude    (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * latitude     (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * depth        (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
    spatial_ref  int64 0
Data variables:
    mlotst       (time, latitude, longitude) float32 ...
    zos          (time, latitude, longitude) float32 ...
    bottomT      (time, latitude, longitude) float32 ...
    sithick      (time, latitude, longitude) float32 ...
    siconc       (time, latitude, longitude) float32 ...
    usi          (time, latitude, longitude) float32 ...
    vsi          (time, latitude, longitude) float32 ...
    thetao       (time, depth, latitude, longitude) float32 ...
    so           (time, depth, latitude, longitude) float32 ...
    uo           (time, depth, latitude, longitude) float32 ...
    vo           (time, depth, latitude, longitude) float32 ...
Attributes: (12/16)
    CDI:                           Climate Data Interface version 1.9.3 (http...
    source:                        MERCATOR GLORYS12V1
    institution:                   Mercator Ocean
    Conventions:                   CF-1.6
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    ...                            ...
    product:                       GLOBAL_REANALYSIS_001_030
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    CDO:                           Climate Data Operators version 1.9.3 (http...
    title:                         Monthly climatology fields for product GLO...
    dataset:                       global-reanalysis-phy-001-030-monthly-clim...

In [5]:
out["mercatorglorys12v1_gl12_mean_1993_2016_04"]['time']

<xarray.DataArray 'time' (time: 1)>
array(['1993-04-16T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1993-04-16
Attributes:
    standard_name:  time
    long_name:      Time (hours since 1950-01-01)
    axis:           T

In [24]:
def reduce_xa_array(xa_array: xa.DataArray, resolution: float=0.01, shape: tuple=None) -> xa.DataArray:
    """TODO: docstring"""

    if not shape:
        reduced_array = array.rio.reproject(array.rio.crs, resolution=0.01)
    else:
        reduced_array = array.rio.reproject(array.rio.crs, shape=shape)

    return reduced_array


def reduce_dict_of_xa_arrays(xa_dict: dict, resolution: float=0.01, shape: tuple=None) -> dict:
    """TODO: docstring
    NB farmed out process to reduce_xa_array function"""
    reduced_dict = {}
    for name, array in tqdm(xa_dict.items()):
        reduced_name = remove_suffix(name) + "_reduced"
        reduced_array = reduce_xa_array(array, resolution, shape)
        
        reduced_dict[reduced_name] = reduced_array

    return reduced_dict

In [34]:
fig, ax = plt.subplots(12, 1)

for i, array in tqdm(enumerate(out.values())):
    # reduced_array = reduce_xa_array(array, resolution=1)
    array["bottomT"].plot(ax=ax[i])

# reduced_array["bottomT"].plot(ax=ax[i])

8it [00:20,  2.51s/it]


RuntimeError: NetCDF: HDF error

Error in callback <function _draw_all_if_interactive at 0x15133b920> (for post_execute):


KeyboardInterrupt: 

In [5]:
fig, ax = plt.subplots(len(out), 1)

for i, array in tqdm(enumerate(out.values())):
    array["mlotst"].plot(ax=ax[i])

ERROR:tornado.general:SEND Error: Host unreachable
12it [00:23,  1.94s/it]


Error in callback <function _draw_all_if_interactive at 0x14e647a60> (for post_execute):


KeyboardInterrupt: 

In [33]:
reduce_xa_array(out["mercatorglorys12v1_gl12_mean_1993_2016_05"]['mlotst'])

KeyboardInterrupt: 

In [32]:
(reduce_xa_array(out["mercatorglorys12v1_gl12_mean_1993_2016_05"]['mlotst'])).plot()

KeyboardInterrupt: 

In [63]:
out["mercatorglorys12v1_gl12_mean_1993_2016_08"].variables.keys()

KeysView(Frozen({'time': <xarray.IndexVariable 'time' (time: 1)>
array(['1993-08-16T12:00:00.000000000'], dtype='datetime64[ns]')
Attributes:
    standard_name:  time
    long_name:      Time (hours since 1950-01-01)
    axis:           T, 'longitude': <xarray.IndexVariable 'longitude' (longitude: 4320)>
array([-180.     , -179.91667, -179.83333, ...,  179.75   ,  179.83333,
        179.91667], dtype=float32)
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X, 'latitude': <xarray.IndexVariable 'latitude' (latitude: 2041)>
array([-80.      , -79.916664, -79.833336, ...,  89.833336,  89.916664,
        90.      ], dtype=float32)
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north
    axis:           Y, 'depth': <xarray.IndexVariable 'depth' (depth: 50)>
array([4.940250e-01, 1.541375e+00, 2.645669e+00, 3.819495e+00, 5.078224e+00,
       6.440614e+00, 7.929560e+00

In [43]:
out["mercatorglorys12v1_gl12_mean_1993_2016_07"].variables["usi"][:][0]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [0.01104770042002201, 0.01110873743891716, 0.011139255948364735,
         ..., 0.010925626382231712, 0.010986663401126862,
         0.011017181910574436],
        [0.0038453321903944016, 0.003875850699841976,
         0.003906369209289551, ..., 0.0036927396431565285,
         0.0037537766620516777, 0.0037842951714992523],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=-32767,
  dtype=float32)

## Ocean Acidification

In [66]:
acids = read_ncs_from_dir("/Volumes/MRes Drive/ocean_acidification")

100%|██████████| 2/2 [00:00<00:00,  2.97it/s]


In [69]:
acids["Aragonite_std_historical"]

<xarray.Dataset>
Dimensions:    (time: 18, lat: 180, lon: 360)
Coordinates:
  * time       (time) float64 1.75e+03 1.85e+03 1.86e+03 ... 2e+03 2.01e+03
Dimensions without coordinates: lat, lon
Data variables:
    aragonite  (time, lat, lon) float64 ...
    longitude  (lat, lon) float64 ...
    latitude   (lat, lon) float64 ...
Attributes:
    title:               Standard deviations of global surface ocean aragonit...
    comment:             This gridded data product contains the standard devi...
    reference:           Jiang, L-Q., J. Dunne, B. R. Carter, J. Tjiputra,\n ...
    Fair_use_statement:  This data product is made freely available\n   to th...
    created_by:          Li-Qing Jiang
    institution:         (a) Cooperative Institute for Satellite Earth System...
    contact:             <Liqing.Jiang@noaa.gov>
    creation_date:       August 14, 2022